<a href="https://colab.research.google.com/github/chicago007/test2/blob/main/%EC%A0%80%EC%9E%A5%ED%85%8C%EC%8A%A4%ED%8A%B8__%EC%A0%84%EB%9E%B5%ED%85%8C%EC%8A%A4%ED%8A%B801.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pykrx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 17.9 MB/s eta 0:00:00


In [ ]:
# prompt: pykrx에서 삼성전자 주가를 가져와서 2024년1월2일~2025년2월7일까지 ema 적용하는 전략에서 최적값을 찾는 코드를 생성해줘

#!pip install pykrx

import pandas as pd
from pykrx import stock
import numpy as np

def calculate_vwema(data, window):
    """Calculates the Volume-Weighted Exponential Moving Average (VWEMA)."""
    typical_price = (data['고가'] + data['저가'] + data['종가']) / 3
    volume_weighted_price = typical_price * data['거래량']

    # Calculate EWMA of volume-weighted price and volume separately
    vwema = volume_weighted_price.ewm(span=window, adjust=False).mean() / data['거래량'].ewm(span=window, adjust=False).mean()

    return vwema

def backtest_strategy(data, short_window, long_window):
    """Backtests the dual VWEMA strategy."""
    data['vwema_short'] = calculate_vwema(data, short_window)
    data['vwema_long'] = calculate_vwema(data, long_window)

    data['signal'] = 0.0
    data.loc[data['vwema_short'] > data['vwema_long'], 'signal'] = 1.0  # Buy signal
    data.loc[data['vwema_short'] < data['vwema_long'], 'signal'] = 0.0  # Sell signal

    data['positions'] = data['signal'].diff()
    data['returns'] = data['종가'].pct_change()
    data['strategy_returns'] = data['returns'] * data['signal'].shift(1)

    total_return = (1 + data['strategy_returns']).cumprod().iloc[-1] - 1
    return total_return

# Fetch Samsung Electronics stock data (2024-01-02 to 2025-02-07)
start_date = "20240102"
end_date = "20250207"
ticker = "005930"  # Samsung Electronics ticker symbol
df = stock.get_market_ohlcv_by_date(start_date, end_date, ticker)


# Optimize dual VWEMA windows
best_short_window = 0
best_long_window = 0
best_return = -1.0

for short_window in range(5, 21):
    for long_window in range(short_window + 5, 51):
        total_return = backtest_strategy(df.copy(), short_window, long_window)
        if total_return > best_return:
            best_return = total_return
            best_short_window = short_window
            best_long_window = long_window

print(f"Optimal Short VWEMA Window: {best_short_window}")
print(f"Optimal Long VWEMA Window: {best_long_window}")
print(f"Maximum Return: {best_return:.4f}")

Optimal Short VWEMA Window: 12
Optimal Long VWEMA Window: 33
Maximum Return: 0.0016


In [ ]:
# !pip install pykrx
!pip install deap

import pandas as pd
from pykrx import stock
import numpy as np
from deap import base, creator, tools, algorithms

def calculate_vwsma(data, window):
    """Calculates the Volume-Weighted Simple Moving Average (VWSMA)."""
    typical_price = (data['고가'] + data['저가'] + data['종가']) / 3
    volume_weighted_price = typical_price * data['거래량']
    vwsma = volume_weighted_price.rolling(window=window).sum() / data['거래량'].rolling(window=window).sum()
    return vwsma

def backtest_strategy(data, individual):
    """Backtests the dual VWSMA strategy using individual parameters."""
    short_window, long_window = individual

    data['vwsma_short'] = calculate_vwsma(data, short_window)
    data['vwsma_long'] = calculate_vwsma(data, long_window)

    data['signal'] = 0.0
    data.loc[data['vwsma_short'] > data['vwsma_long'], 'signal'] = 1.0  # Buy signal
    data.loc[data['vwsma_short'] < data['vwsma_long'], 'signal'] = 0.0  # Sell signal

    data['positions'] = data['signal'].diff()
    data['returns'] = data['종가'].pct_change()
    data['strategy_returns'] = data['returns'] * data['signal'].shift(1)

    total_return = (1 + data['strategy_returns']).cumprod().iloc[-1] - 1
    return total_return,


# Fetch Samsung Electronics stock data (2024-01-02 to 2025-02-07)
start_date = "20240102"
end_date = "20250207"
ticker = "005930"  # Samsung Electronics ticker symbol
df = stock.get_market_ohlcv_by_date(start_date, end_date, ticker)

# Define the fitness function
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_int", np.random.randint, 5, 50)  # Window range: 5 to 50
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, 2)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Define evaluation, selection, mutation, and crossover functions
toolbox.register("evaluate", lambda individual: backtest_strategy(df.copy(), individual))
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=5, up=50, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

# Run the genetic algorithm
population = toolbox.population(n=50)  # Population size
ngen = 50  # Number of generations
cxpb = 0.5  # Crossover probability
mutpb = 0.2  # Mutation probability

# Run the evolution
pop, logbook = algorithms.eaSimple(population, toolbox, cxpb, mutpb, ngen, stats=None, halloffame=None, verbose=False)

# Get the best individual (optimal windows)
best_individual = tools.selBest(pop, 1)[0]
best_short_window, best_long_window = best_individual

print(f"Optimal Short VWSMA Window: {best_short_window}")
print(f"Optimal Long VWSMA Window: {best_long_window}")
# Calculate the maximum return using the best individual
best_return, = backtest_strategy(df.copy(), best_individual)
print(f"Maximum Return: {best_return:.4f}")

/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Optimal Short VWSMA Window: 14
Optimal Long VWSMA Window: 22
Maximum Return: 0.0360
